In [1]:
import scipy.io as sio
import numpy as np
import keras
import sklearn.metrics
import math
from keras.layers import Input, Add, Dense, Activation, ZeroPadding3D, BatchNormalization, Flatten, Conv3D, AveragePooling3D, MaxPooling3D, GlobalMaxPooling3D
from keras.models import Model
from keras.utils import plot_model
from keras.initializers import glorot_uniform
from sklearn.preprocessing import OneHotEncoder
from keras import regularizers

import warnings
warnings.filterwarnings('ignore')

from matplotlib.pyplot import imshow
%matplotlib inline

Using TensorFlow backend.


In [2]:
def pick_class(Class , Cube_size , Data, Gt, small_segmented_1, small_seg_gt_1, overlap_ratio, ch):
    indx_class = np.where(Gt == Class)
    all_indx = [[indx_class[0][i],indx_class[1][i]] for i in range(len(indx_class[0])) if len(Gt)-np.ceil(Cube_size/2)>indx_class[0][i]>np.ceil(Cube_size/2) and len(Gt[0])-np.ceil(Cube_size/2)>indx_class[1][i]>np.ceil(Cube_size/2)]
    lst = []
    small_segmented_1.append(np.array(Data[:ch, all_indx[0][0]-int(Cube_size/2):all_indx[0][0] + int(Cube_size/2), (all_indx[0][1]-int(Cube_size/2)):all_indx[0][1]+int(Cube_size/2)]))
    small_seg_gt_1.append(Class)
    lst.append([all_indx[0][0], all_indx[0][1]])
    for i in range(1, len(all_indx)):
        dist = []
        for k in range(len(lst)):
            d = math.sqrt((all_indx[i][0]-lst[k][0])**2 + (all_indx[i][1]-lst[k][1])**2)
            dist.append(d)
        if np.min(dist) > int(Cube_size*(1-overlap_ratio)):
            small_segmented_1.append(np.array(Data[:ch, all_indx[i][0]-int(Cube_size/2):all_indx[i][0] + int(Cube_size/2), (all_indx[i][1]-int(Cube_size/2)):all_indx[i][1]+int(Cube_size/2)]))
            small_seg_gt_1.append(Class)
            lst.append([all_indx[i][0], all_indx[i][1]])
    return small_segmented_1, small_seg_gt_1, lst

def pick_n_class(range_of_class, Cube_size, Data, Gt, small_segmented_1, small_seg_gt_1, overlap_ratio, ch):
    class_len = []
    for i in range_of_class:
        
        small_segmented_1, small_seg_gt_1, lst = pick_class(i, Cube_size, Data, Gt, small_segmented_1, small_seg_gt_1, overlap_ratio, ch)
        class_len.append(len(lst))
    small_segmented_1 = np.array(small_segmented_1)
    small_seg_gt_1 = np.array(small_seg_gt_1)
    return small_segmented_1, small_seg_gt_1, class_len



def train_test_split(percentage, class_len, Data, Gt):
    Xtrain = []
    Xtest = []
    Ytrain = []
    Ytest = []
    class_division = [0]
    c = 0
    for i in range(len(class_len)):
        class_division.append(int(class_len[i]*(percentage/100)) + c)
        class_division.append(class_len[i]+c)
        c = class_len[i]+c
    for i in range(1, len(class_division)):
        if i%2!=0:
            for j in range(class_division[i-1], class_division[i]):
                Xtrain.append(Data[j])
                Ytrain.append(Gt[j])
        else:
            for k in range(class_division[i-1], class_division[i]):
                Xtest.append(Data[k])
                Ytest.append(Gt[k])    
    Xtrain = np.array(Xtrain)
    Xtest = np.array(Xtest)
    Ytrain = np.array(Ytrain)
    Ytest = np.array(Ytest)
    s = np.arange(Xtrain.shape[0])
    np.random.shuffle(s)
    Xtrain = Xtrain[s]
    Ytrain = Ytrain[s]
    s = np.arange(Xtest.shape[0])
    np.random.shuffle(s)
    Xtest = Xtest[s]
    Ytest = Ytest[s]
    Xtrain = np.expand_dims(Xtrain,axis=4) 
    Xtest = np.expand_dims(Xtest,axis=4) 
    Xtrain.shape, Xtest.shape, Ytrain.shape, Ytest.shape
    values,counts = np.unique(Ytest, return_counts=True)
    print("Total samples per class: " + str(class_len) + ", Total number of samples is " + str(np.sum(class_len))+'.\n')
    print("unique classes in Ytest: " + str(values) + ", Total number of samples in Ytest is " + str(np.sum(counts))+'.\n'
          +"number of samples per class in Ytest: " + str(counts) + '\n')
    
    onehot_encoder = OneHotEncoder(sparse=False)
    Ytrain = Ytrain.reshape(len(Ytrain), 1)
    Ytest = Ytest.reshape(len(Ytest),1)
    Ytrain = onehot_encoder.fit_transform(Ytrain)
    Ytest = onehot_encoder.fit_transform(Ytest)
    
    return Xtrain, Xtest, Ytrain, Ytest, class_len, counts

def prepare_data_for_training(range_of_class, Cube_size, Data , Gt, small_segmented_1, small_seg_gt_1, percentage, overlap_ratio, ch):
    small_segmented_1, small_seg_gt_1, class_len = pick_n_class(range_of_class, Cube_size, Data, Gt, small_segmented_1, small_seg_gt_1, overlap_ratio, ch)
    Xtrain, Xtest, Ytrain, Ytest, class_len, counts = train_test_split(percentage, class_len, small_segmented_1, small_seg_gt_1)
    return Xtrain, Xtest, Ytrain, Ytest, class_len, counts

def SR_Unit(X, filters):
    # Save the input value
    X_shortcut = X
    l2_= 0.001
    X = Conv3D(filters, (1, 1, 1), kernel_regularizer=regularizers.l2(l2_),padding = "same", name='SR_Unit_'+str(filters)+'_conv_1')(X)
    X = BatchNormalization(axis = 4, name='SR_Unit_'+str(filters)+'_bn_1')(X)
    X = Activation('relu', name='SR_Unit_'+str(filters)+'_activation_1')(X)

    X = Conv3D(filters, kernel_size = (1, 3, 3), kernel_regularizer=regularizers.l2(l2_), padding = 'same', name='SR_Unit_'+str(filters)+'_conv_2')(X)
    X = BatchNormalization(axis = 4, name='SR_Unit_'+str(filters)+'_bn_2')(X)
    X = Activation('relu', name='SR_Unit_'+str(filters)+'_activation_2')(X)

    X = Conv3D(filters, kernel_size = (3, 1, 1), kernel_regularizer=regularizers.l2(l2_), padding = "same", name='SR_Unit_'+str(filters)+'_conv_3')(X)
    X = BatchNormalization(axis = 4, name='SR_Unit_'+str(filters)+'_bn_3')(X)
    X = Activation('relu', name='SR_Unit_'+str(filters)+'_activation_3')(X)

    X = Conv3D(2*filters, kernel_size = (1, 1, 1), kernel_regularizer=regularizers.l2(l2_), padding = "same", name='SR_Unit_'+str(filters)+'_conv_4')(X)
    X = BatchNormalization(axis = 4, name='SR_Unit_'+str(filters)+'_bn_4')(X)

    X_shortcut = Conv3D(filters = 2*filters, kernel_size = (3, 3, 3), padding = 'same', kernel_regularizer=regularizers.l2(l2_), name='SR_Unit_'+str(filters)+'_conv_5')(X_shortcut)
    X_shortcut = BatchNormalization(axis = 4, name='SR_Unit_'+str(filters)+'_bn_5')(X_shortcut)

    X = Add()([X, X_shortcut])
    X = Activation('relu', name='SR_Unit_'+str(filters)+'_activation_5')(X)
    
    return X

def feature_extraction(Sample):
    X = Sample
    
    X = Conv3D(32, (8, 3, 3), kernel_regularizer=regularizers.l2(0.001), strides=(2,2,2), name='conv_0')(X)
    X = BatchNormalization(axis=4, name='bn_conv_0')(X)
    X = Activation('relu')(X)
    X = MaxPooling3D((1, 2, 2), strides=None, name='MaxPooling_0')(X)
    
    # Stage 2
    F1 = 16
    X = SR_Unit(X, filters=F1)
    
    F2 = 32
    X = SR_Unit(X, filters=F2)

#     F3 = 64
#     X = SR_Unit(X, filters=F3)

#     F4 = 128
#     X = SR_Unit(X, filters=F4)

    return X

def model(input_shape=(103, 50, 50, 1), classes=9):
    X_input = Input(input_shape)
    
    X = feature_extraction(X_input)
    
    X = GlobalMaxPooling3D()(X)
    X = Dense(256, input_dim=X.shape, activation='relu', name='fc_256', kernel_initializer = glorot_uniform(seed=0))(X)
    X = Dense(classes, input_dim=X.shape, activation='softmax', name='fc' + str(classes), kernel_initializer = glorot_uniform(seed=0))(X)
    
    model = Model(inputs = X_input, outputs = X, name="3D-SRNet")

    return model

# Pavia

In [3]:
uPavia = sio.loadmat('C:\\Users\\de991521\\Desktop\\EE_297_PROJECT\\pavia\\PaviaU.mat')
gt_uPavia = sio.loadmat('C:\\Users\\de991521\\Desktop\\EE_297_PROJECT\\pavia\\PaviaU_gt.mat')
data_PV = uPavia['paviaU']
gt_PV = gt_uPavia['paviaU_gt']
print(data_PV.shape, gt_PV.shape)
data_PV = np.moveaxis(data_PV, 2, 0)
print(data_PV.shape, gt_PV.shape)

(610, 340, 103) (610, 340)
(103, 610, 340) (610, 340)


In [4]:
values,counts = np.unique(gt_PV, return_counts=True)
print(values,counts)
range_of_class = list(values)
if 0 in range_of_class:
    range_of_class.pop(0)
range_of_class

[0 1 2 3 4 5 6 7 8 9] [164624   6631  18649   2099   3064   1345   5029   1330   3682    947]


[1, 2, 3, 4, 5, 6, 7, 8, 9]

In [5]:
Xtrain, Xtest, Ytrain, Ytest, class_len, counts = prepare_data_for_training(range_of_class = range_of_class, Cube_size = 25, Data = data_PV , Gt = gt_PV, small_segmented_1 = [], small_seg_gt_1 = [], percentage = 80, overlap_ratio = 0.8, ch = 103)

Total samples per class: [475, 881, 131, 347, 104, 312, 95, 343, 104], Total number of samples is 2792.

unique classes in Ytest: [1 2 3 4 5 6 7 8 9], Total number of samples in Ytest is 562.
number of samples per class in Ytest: [ 95 177  27  70  21  63  19  69  21]



In [6]:
Xtrain.shape, Xtest.shape, Ytrain.shape, Ytest.shape

((2230, 103, 24, 24, 1), (562, 103, 24, 24, 1), (2230, 9), (562, 9))

In [7]:
model_1 = model(input_shape = Xtrain[0].shape, classes = len(range_of_class))
model_1.summary()

Instructions for updating:
Colocations handled automatically by placer.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 103, 24, 24,  0                                            
__________________________________________________________________________________________________
conv_0 (Conv3D)                 (None, 48, 11, 11, 3 2336        input_1[0][0]                    
__________________________________________________________________________________________________
bn_conv_0 (BatchNormalization)  (None, 48, 11, 11, 3 128         conv_0[0][0]                     
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 48, 11, 11, 3 0           bn_conv_0[0][0]                  
_____________________________________

In [ ]:
model_1.compile(optimizer= keras.optimizers.SGD(lr=0.01, decay=1e-5, momentum=0.9, nesterov=True), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model_1.fit(Xtrain, Ytrain, epochs = 50, batch_size = 32, validation_data=(Xtest,Ytest))

Train on 2230 samples, validate on 562 samples
Epoch 1/50
2230/2230 [==============================] - 133s 60ms/step - loss: 11.2540 - categorical_accuracy: 0.3076 - val_loss: 11.4099 - val_categorical_accuracy: 0.3149
Epoch 2/50
2230/2230 [==============================] - 127s 57ms/step - loss: 11.3929 - categorical_accuracy: 0.3157 - val_loss: 11.3998 - val_categorical_accuracy: 0.3149
Epoch 3/50
2230/2230 [==============================] - 141s 63ms/step - loss: 11.3829 - categorical_accuracy: 0.3157 - val_loss: 11.3899 - val_categorical_accuracy: 0.3149
Epoch 4/50
2230/2230 [==============================] - 138s 62ms/step - loss: 11.3731 - categorical_accuracy: 0.3157 - val_loss: 11.3803 - val_categorical_accuracy: 0.3149
Epoch 5/50
2230/2230 [==============================] - 135s 60ms/step - loss: 11.3636 - categorical_accuracy: 0.3157 - val_loss: 11.3710 - val_categorical_accuracy: 0.3149
Epoch 6/50
2230/2230 [==============================] - 134s 60ms/step - loss: 11.3544 -

In [ ]:
preds = model_1.evaluate(Xtest, Ytest)
# print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

In [ ]:
y_pred = model_1.predict(Xtest, verbose=1)

In [ ]:
confusion_matrix = sklearn.metrics.confusion_matrix(np.argmax(Ytest, axis=1), np.argmax(y_pred, axis=1))
confusion_matrix

In [ ]:
counts

In [ ]:
model_1.layers[0:36]

In [ ]:
model_new = Model(input=model_1.layers[0].input, output=model_1.layers[35].output)
model_new.save("model_pavia_overlap_80.h5")
model_new.summary()

# Salinas

In [ ]:
uSalinas = sio.loadmat('C:\\Users\\de991521\\Desktop\\EE_297_PROJECT\\Salinas\\Salinas_corrected.mat')
gt_uSalinas = sio.loadmat('C:\\Users\\de991521\\Desktop\\EE_297_PROJECT\\Salinas\\Salinas_gt.mat')
data_SA = uSalinas['salinas_corrected']
gt_SA = gt_uSalinas['salinas_gt']
print(data_SA.shape, gt_SA.shape)
data_SA = np.moveaxis(data_SA, 2, 0)
print(data_SA.shape, gt_SA.shape)

In [ ]:
values,counts = np.unique(gt_SA, return_counts=True)
print(values,counts)
range_of_class = list(values)
if 0 in range_of_class:
    range_of_class.pop(0)
range_of_class

In [ ]:
Xtrain, Xtest, Ytrain, Ytest, class_len, counts = prepare_data_for_training(range_of_class = range_of_class, Cube_size = 25, Data = data_SA , Gt = gt_SA, small_segmented_1 = [], small_seg_gt_1 = [], percentage = 80, overlap_ratio = 0.8, ch=103)

In [ ]:
Xtrain.shape, Xtest.shape, Ytrain.shape, Ytest.shape

In [ ]:
model_1 = model(input_shape = Xtrain[0].shape, classes = len(range_of_class))

In [ ]:
model_1.summary()

In [ ]:
model_1.compile(optimizer= keras.optimizers.SGD(lr=0.001, decay=1e-5, momentum=0.9), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model_1.fit(Xtrain, Ytrain, epochs = 45, batch_size = 64)

In [ ]:
model_1.layers[0:35]

In [ ]:
model_new = Model(input=model_1.layers[0].input, output=model_1.layers[35].output)
model_new.save("model_salinas.h5")
model_new.summary()

In [ ]:
preds = model_1.evaluate(Xtest, Ytest)
# print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

In [ ]:
y_pred = model_1.predict(Xtest, verbose=1)

In [ ]:
confusion_matrix = sklearn.metrics.confusion_matrix(np.argmax(Ytest, axis=1), np.argmax(y_pred, axis=1))
confusion_matrix

In [ ]:
counts

# Indian Pines

In [ ]:
uIndian_pines = sio.loadmat('C:\\Users\\de991521\\Desktop\\EE_297_PROJECT\\Indian Pines\\Indian_pines_corrected.mat')
gt_uIndian_pines = sio.loadmat('C:\\Users\\de991521\\Desktop\\EE_297_PROJECT\\Indian Pines\\Indian_pines_gt.mat')
data_IN = uIndian_pines['indian_pines_corrected']
gt_IN = gt_uIndian_pines['indian_pines_gt']
print(data_IN.shape, gt_IN.shape)
data_IN = np.moveaxis(data_IN, 2, 0)
print(data_IN.shape, gt_IN.shape)

In [ ]:
values,counts = np.unique(gt_IN, return_counts=True)
print(values,counts)
range_of_class = list(values)
if 0 in range_of_class:
    range_of_class.pop(0)
range_of_class

In [ ]:
Xtrain, Xtest, Ytrain, Ytest, class_len, counts = prepare_data_for_training(range_of_class = range_of_class, Cube_size = 10, Data = data_IN , Gt = gt_IN, small_segmented_1 = [], small_seg_gt_1 = [], percentage = 80)

In [ ]:
class_len

In [ ]:
Xtrain.shape, Xtest.shape, Ytrain.shape, Ytest.shape

In [ ]:
model_1 = model(input_shape = Xtrain[0].shape, classes = len(range_of_class))

In [ ]:
model_1.summary()

In [ ]:
model_1.compile(optimizer= keras.optimizers.SGD(lr=0.01, decay=1e-5, momentum=0.9), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model_1.fit(Xtrain,model_1.layers[0:35]

In [ ]:
model_1.layers[0:35]

In [ ]:
model_new = Model(input=model_1.layers[0].input, output=model_1.layers[35].output)
model_new.save("model_indian.h5")
model_new.summary()

In [ ]:
preds = model_1.evaluate(Xtest, Ytest)
# print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

In [ ]:
y_pred = model_1.predict(Xtest, verbose=1)

In [ ]:
confusion_matrix = sklearn.metrics.confusion_matrix(np.argmax(Ytest, axis=1), np.argmax(y_pred, axis=1))
confusion_matrix

In [ ]:
counts